In [2]:
import ee, geemap, datetime as dt

In [ ]:
ee.Initialize(project="flood-predictor-458709")

# Bounding box roughly covering Sindh
sindh_coords = [[[69.36930566940416, 28.491271735833283],
          [68.47941309127916, 28.47195842642833],
          [68.34757715377916, 28.3463359106437],
          [68.12785059127916, 28.21088376360882],
          [67.84220606002916, 27.95887386849454],
          [67.44669824752916, 27.871502096599],
          [67.24894434127916, 27.64789865441828],
          [67.16105371627916, 27.375067747692672],
          [67.15006738815416, 26.856795208062252],
          [67.18302637252916, 26.50339897787297],
          [67.39176660690416, 26.208068833533147],
          [67.44669824752916, 25.823017465134107],
          [67.16105371627916, 25.397017986935314],
          [67.12809473190416, 25.23812034267632],
          [66.98527246627916, 25.03920590911963],
          [66.74357324752916, 24.979468485309294],
          [66.65568262252916, 24.85990659856756],
          [66.85343652877916, 24.84993787926005],
          [67.02921777877916, 24.790108702044094],
          [67.19401270065416, 24.610448167399934],
          [67.28190332565416, 24.420526166703684],
          [67.32584863815416, 24.250353244154304],
          [67.44669824752916, 24.03982500031995],
          [67.60050684127916, 23.87919042293869],
          [67.75431543502916, 23.81890093669589],
          [67.98502832565416, 23.748527794610634],
          [68.21574121627916, 23.68817767030865],
          [68.34757715377916, 23.98964820004025],
          [68.76505762252916, 23.969572000697074],
          [68.77604395065416, 24.29041448712888],
          [69.21054350907603, 24.261840527523468],
          [69.6670214863455, 24.292056581676597],
          [69.71403002286215, 24.216742897925894],
          [69.87081327525628, 24.181436366260744],
          [70.00562342358802, 24.19614369123337],
          [70.07460327137314, 24.29093981141138],
          [70.56141708098181, 24.466418133542536],
          [70.63472428470699, 24.204100054763565],
          [70.8177282272066, 24.302157509305587],
          [70.93492204158476, 24.37989590371818],
          [71.08151843342519, 24.428764003333672],
          [71.0744716593636, 24.677576404550244],
          [70.84043982818369, 25.147951066380926],
          [70.66641732292094, 25.42360166566876],
          [70.62370484096166, 25.699111132383642],
          [70.41165846802771, 25.65034336300033],
          [70.22145618335809, 25.759740585590844],
          [70.05181099656994, 26.06207345751372],
          [70.1535035219853, 26.1936301910018],
          [70.16049507599844, 26.387482471054657],
          [70.1014902865915, 26.541960098456148],
          [69.9993555879304, 26.57509017665521],
          [69.83124811223924, 26.588440729322624],
          [69.4506190063056, 26.83120223284854],
          [69.5704928853013, 27.163223493069317],
          [69.95672415471721, 27.54644972738804],
          [70.08453019526463, 27.71768632998689],
          [70.1595535018085, 27.881185710690666],
          [69.98282645814479, 27.989720403411862],
          [69.82215474802067, 28.31374404351952],
          [69.816603058706, 28.368010725946274],
          [69.73408392749116, 28.465760399532567],
          [69.60189753349103, 28.477599319727837],
          [69.57075317347648, 28.513450205058234],
          [69.51121756623884, 28.51929256412606],
          [69.36930566940416, 28.491271735833283]]]


region = ee.Geometry.Polygon(sindh_coords)

# 2010‑01‑01 00:00 to 2025‑08‑02 00:00
start = ee.Date('2010-01-01T00:00')
end   = ee.Date('2025-08-02T00:00')

# Create an interactive 
Map = geemap.Map(center=[28, 66], zoom=6)

# Add the polygon to the map
Map.addLayer(region, {'color': 'red'}, 'Sindh Border')

# Display the map
Map.to_html("sindh_map.html")

In [6]:
imerg = (ee.ImageCollection('NASA/GPM_L3/IMERG_V07')
           .filterDate(start, end)
           .filterBounds(region)
           .select('precipitationCal')) 

In [12]:
n_days = end.difference(start, 'day').subtract(1)
zero_rain = ee.Image.constant(0).rename('precipitationCal')

def one_day(n):
    day_start = start.advance(n, 'day')
    day_end   = day_start.advance(1, 'day')
    day_imgs  = imerg.filterDate(day_start, day_end)

    daily_mm = ee.Algorithms.If(
        day_imgs.size().gt(0),
        day_imgs.sum(),           # collection has data
        zero_rain                 # fallback image
    )
    daily_mm = ee.Image(daily_mm).multiply(0.5)      # mm depth

    mean_mm = daily_mm.reduceRegion(
        ee.Reducer.mean(), region, 1e4, bestEffort=True).get('precipitationCal')

    return ee.Feature(None, {
        'time': day_start.format('YYYY-MM-dd'),
        'rain_mm': mean_mm               # will be 0 for missing days
    })


In [ ]:
days = ee.List.sequence(0, n_days)
daily_fc  = ee.FeatureCollection(days.map(one_day))

# --- 4A. Quick in‑notebook grab (good for ≤ ~5 000 rows) -----------------------
df = geemap.ee_to_df(daily_fc)     # pulls everything into a pandas DataFrame
df.to_csv('sindh_daily_precip_2010-2025.csv', index=False)
print(df.head())
print("Saved", len(df), "rows to CSV.")


   rain_mm        time
0        0  2025-08-01
Saved 1 rows to CSV.


In [14]:
print('IMERG scenes in your span:',
      imerg.size().getInfo())                       # should be > 0

# How many days did the mapper actually emit?
test_fc = ee.FeatureCollection(days.map(one_day))
print('Rows before filter:',   test_fc.size().getInfo())
print('Rows after notNull():', daily_fc.size().getInfo())


IMERG scenes in your span: 0
Rows before filter: 1
Rows after notNull(): 1
